# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [5]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [6]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [7]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [8]:
f = lambda x, y: x ** y
foldl(f, 2, [2, 2, 3]), \
foldr(f, 1, [2, 2, 3])

(4096, 256)

In [9]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [10]:
def flip(f):
    def flipArgs(x, y):
        return f(y, x)
    return flipArgs

In [14]:
def foldl2(f, x0, lst):
    return foldr(flip(f), x0, lst[::-1])

In [15]:
def foldr2(f, x0, lst):
    return foldl(flip(f), x0, lst[::-1])

In [16]:
#сумма
foldl2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [17]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [18]:
foldl2(lambda x, y: x - y, 0, [1, 2, 3])

-6

In [19]:
foldr2(lambda x, y: x - y, 0, [1, 2, 3])

2

In [ ]:
# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [16]:
def check_subseq(a, d):
    for ch in a:
        if d.get(ch) == None:
            return False
        if d.get(ch) > 0:
            d[ch] = d[ch] - 1
            if d.get(ch) == 0:
                d.pop(ch)
    if not d:
        return True
    else:
        return False

In [17]:
from collections import Counter
def check_inv(a, b):
    d = Counter(b)
    for i, c in enumerate(a):
        if (check_subseq(a[i:i+len(d)], d.copy())):
            return True
    return False

- n = length(a)
- m = length(b)
- O(n*m)

In [18]:
check_inv("rorot", "rot"), \
check_inv("rorot", "tor"), \
check_inv("abcrotm", "tro")

(True, True, True)

In [19]:
check_inv("fas", "qqwweea")

False

In [20]:
check_inv("adb", "ab")

False

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [84]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left: 
            yield from self.left
        if (self.left is None) and (self.right is None):
            yield self.value
        if self.right: 
            for node in self.right.__iter__():
                yield node
    
    def __str__(self, depth=0):
        ret = ""

        if self.right != None:
            ret += self.right.__str__(depth + 1)

        ret += "\n" + ("    "*depth) + str(self.value)

        if self.left != None:
            ret += self.left.__str__(depth + 1)

        return ret
    
    def __repr__(self):
        return '(' + repr(self.left) + ',' + \
      str(self.value) + ',' + repr(self.right) + ')'
    

In [85]:
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))
list(tree) == [3, 4, 2]

True

In [86]:
print(tree)


    2
0
        4
    1
        3


In [87]:
tree.__repr__

<bound method Tree.__repr__ of (((None,3,None),1,(None,4,None)),0,(None,2,None))>

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [91]:
OPERATORS = {'+': (1, lambda x, y: x + y), 
             '-': (1, lambda x, y: x - y),
             '*': (2, lambda x, y: x * y),
             '/': (2, lambda x, y: x / y)}

In [106]:
def parse(raw_string):
        number = ''
        for s in raw_string:
            if s in '1234567890':
                number += s  
            elif number: 
                yield int(number) 
                number = ''
            if s in OPERATORS or s in "()":
                yield s 
        if number: 
            yield int(number)

In [107]:
def to_polish_notation(parsed_formula):
        stack = []  
        for token in parsed_formula:
            if token in OPERATORS: 
                while stack and stack[-1] != "(" and OPERATORS[token][0] <= OPERATORS[stack[-1]][0]:
                    yield stack.pop()
                stack.append(token)
            elif token == ")":
                while stack:
                    x = stack.pop()
                    if x == "(":
                        break
                    yield x
            elif token == "(":
                stack.append(token)
            else:
                yield token
        while stack:
            yield stack.pop()

In [108]:
def calcPolish(polish):
        stack = []
        for token in polish:
            if token in OPERATORS:
                y, x = stack.pop(), stack.pop()
                stack.append(OPERATORS[token][1](x, y))
            else:
                stack.append(token)
        return stack[0]

In [109]:
def calc(expr):
    return calcPolish(to_polish_notation(parse(expr)))

calc('2 * (15 - 3 * 4) - 2') == 4

True

In [110]:
calc('1+2+3+4+5+6+7+8+9')

45

In [111]:
calc('(9 * 10 * 11) / 2')

495.0